In [39]:
import numpy as np
import pandas as pd
from data_pre import data_normalize
import warnings
warnings.filterwarnings('ignore')
d = pd.read_excel(r"D:\program\pycharm\model\Data\新训练.xlsx", sheet_name='Sheet2')
# X = df.iloc[:, 1:]
# y = df.iloc[:, 0]
data = data_normalize(d, 'WD')
df = data.fillna_by_random_fix()
X = df.iloc[:, 1:]
y = df.iloc[:, 0]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
aver = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train-aver)/std
X_test = (X_test-aver)/std

In [40]:
df

,WD,WLC,WLA,WIA,TD,TI,UCS,MTL,TN,ATI,TBD
0,1,1,0.000,0.000,1.360,1.290,52.64,0.200,2.0,0.68,11.0
1,4,1,10.100,12.350,99.120,26.890,10.20,0.500,7.0,0.55,20.0
2,3,3,8.900,10.230,63.440,18.640,32.00,1.200,15.0,0.32,40.0
3,2,2,4.300,6.010,35.660,11.560,50.56,1.700,18.0,0.10,350.0
4,2,1,0.000,0.000,2.360,3.570,60.25,1.400,14.0,0.18,221.0
...,...,...,...,...,...,...,...,...,...,...,...
146,2,2,16.858,8.028,56.476,10.459,11.94,0.935,17.0,0.11,180.0
147,4,3,7.214,3.461,65.889,25.101,21.86,1.490,12.0,0.17,204.0
148,3,3,22.491,2.869,30.330,11.504,16.99,1.400,8.0,0.32,309.0
149,2,4,24.662,6.223,89.944,21.963,7.29,0.980,6.0,0.56,192.0


In [46]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)             # 训练

    print ("训练集精度:")   # 训练集精度
    print (clf.score(X_train, y_train))

    print ("\n测试集精度:")    # 测试集精度
    print (clf.score(X_test, y_test))

    y_pred = clf.predict(X_test)

    print ("\n分类识别报告:")      # 分类识别报告
    print (metrics.classification_report(y_test, y_pred))

    # print ("\n混淆矩阵:")
    # print (metrics.confusion_matrix(y_test, y_pred))
params = {
    'n_estimators': 200, # 弱分类器的个数
    'max_depth': 2,       # 弱分类器（CART回归树）的最大深度
    'min_samples_split': 3, # 分裂内部节点所需的最小样本数
    'learning_rate': 0.02,  # 学习率
    'loss': 'deviance',
    'subsample': 0.8,
    'random_state': 1
}
clf = GradientBoostingClassifier(**params)
train_and_evaluate(clf, X_train, X_test, y_train, y_test)

训练集精度:
0.9833333333333333

测试集精度:
0.7741935483870968

分类识别报告:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.92      0.85      0.88        13
           3       0.56      0.71      0.63         7
           4       0.75      0.75      0.75         8
           5       1.00      1.00      1.00         2

    accuracy                           0.77        31
   macro avg       0.64      0.66      0.65        31
weighted avg       0.77      0.77      0.77        31


In [44]:
from sklearn.model_selection import cross_val_score
params = {
    'n_estimators': 400, # 弱分类器的个数
    'max_depth': 2,       # 弱分类器（CART回归树）的最大深度
    'min_samples_split': 3, # 分裂内部节点所需的最小样本数
    'learning_rate': 0.02,  # 学习率
    'loss': 'deviance',
    'subsample': 0.8,
    'random_state': 1
}
clf = GradientBoostingClassifier(**params)
scores = cross_val_score(clf, X, y, scoring='accuracy', cv=10)
scores

array([0.8125    , 0.66666667, 0.66666667, 1.        , 0.8       ,
       0.86666667, 0.8       , 0.66666667, 0.73333333, 0.8       ])

In [45]:
scores.mean()

0.78125

In [22]:
learning_rate = [0.01, 0.015, 0.02, 0.025]
for n in learning_rate:
    params = {
    'n_estimators': 150, # 弱分类器的个数
    'max_depth': 2,       # 弱分类器（CART回归树）的最大深度
    'min_samples_split': 3, # 分裂内部节点所需的最小样本数
    'learning_rate': 0.02,  # 学习率
    'loss': 'deviance',
    'subsample': 0.8,
    'random_state': 1
}
    clf = GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)
    print(n)
    print ("训练集精度:")   # 训练集精度
    print (clf.score(X_train, y_train))

    print ("\n测试集精度:")    # 测试集精度
    print (clf.score(X_test, y_test))
    print('-' * 50)

0.01
训练集精度:
0.9

测试集精度:
0.7419354838709677
--------------------------------------------------
0.015
训练集精度:
0.9416666666666667

测试集精度:
0.7419354838709677
--------------------------------------------------
0.02
训练集精度:
0.9916666666666667

测试集精度:
0.8064516129032258
--------------------------------------------------
0.025
训练集精度:
1.0

测试集精度:
0.7741935483870968
--------------------------------------------------


In [24]:
y_pred = clf.predict(X_test)
# X_train[y_test != y_pred]
y_pred = pd.DataFrame(y_pred)
y_pred.index=y_test.index
df_ = pd.concat([X_test, y_pred, y_test], axis=1)

In [25]:
df_.loc[df_[df_[0] != df_['WD']].index.tolist()]

,WLC,WLA,WIA,TD,TI,UCS,MTL,TN,ATI,TBD,0,WD
39,0.654106,0.010462,0.633426,1.054838,0.217651,0.172064,0.632948,-0.736120,-0.333237,-0.065610,4,3
1,-1.109776,0.616968,2.557656,2.076732,2.405468,-1.251528,-1.712076,-1.928217,1.776612,-2.147040,3,4
144,0.654106,0.452771,0.476103,-0.378537,-0.483824,-1.641553,0.336974,0.575187,2.520470,0.036199,4,5
21,1.536047,0.291717,1.896398,2.196002,1.951367,-0.946008,-0.619249,-2.166636,1.830711,-2.079167,3,2
24,0.654106,0.024603,1.608077,0.808568,0.863765,0.194520,0.314207,0.217558,0.045454,0.499996,3,2
118,-0.227835,0.605183,-0.398575,-1.264044,-0.630426,-1.606096,-0.248144,-3.120314,2.479895,0.273753,2,1


In [27]:
df.iloc[67], df.iloc[21]

(WD       2.000
 WLC      4.000
 WLA      8.030
 WIA     10.240
 TD     102.360
 TI      23.650
 UCS     15.370
 MTL      0.954
 TN       6.000
 ATI      0.560
 TBD    192.000
 Name: 67, dtype: float64,
 WD       2.00
 WLC      4.00
 WLA      8.03
 WIA     10.24
 TD     102.36
 TI      23.65
 UCS     15.37
 MTL      0.98
 TN       6.00
 ATI      0.56
 TBD     26.00
 Name: 21, dtype: float64)

In [9]:
d[d['WLC']==4]

,WD,WLC,WLA,WIA,TD,TI,UCS,MTL,TN,ATI,TBD
5,4,4,12.350,11.680,106.530,24.280,4.66,1.700,17.0,0.120,290.0
6,4,4,10.320,8.670,70.230,17.830,17.58,0.400,9.0,0.570,42.0
10,4,4,12.660,11.350,106.230,27.340,5.24,1.860,17.0,0.139,365.0
16,3,4,10.250,8.640,65.560,16.240,15.54,1.200,9.0,0.590,159.0
17,3,4,6.450,4.870,34.870,9.560,32.57,1.400,12.0,0.380,256.0
21,2,4,8.030,10.240,102.360,23.650,15.37,0.980,6.0,0.560,26.0
44,3,4,7.880,6.570,62.470,8.380,39.64,1.560,16.0,0.180,194.0
51,4,4,12.350,11.680,106.530,24.280,4.66,NaN,NaN,NaN,NaN
52,4,4,10.320,8.670,70.230,17.830,17.58,NaN,NaN,NaN,NaN
56,4,4,12.660,11.350,106.230,27.340,5.24,NaN,NaN,NaN,NaN


In [16]:
import numpy as np
import pandas as pd
from data_pre import data_normalize
import warnings
warnings.filterwarnings('ignore')
d = pd.read_csv(r"D:\program\pycharm\model\Data\out.csv")
# X = df.iloc[:, 1:]
# y = df.iloc[:, 0]
data = data_normalize(d, 'WD')
df = data.fillna_by_random_fix()
X = df.iloc[:, 1:]
y = df.iloc[:, 0]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
aver = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train-aver)/std
X_test = (X_test-aver)/std
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)             # 训练

    print ("训练集精度:")   # 训练集精度
    print (clf.score(X_train, y_train))

    print ("\n测试集精度:")    # 测试集精度
    print (clf.score(X_test, y_test))

    y_pred = clf.predict(X_test)

    print ("\n分类识别报告:")      # 分类识别报告
    print (metrics.classification_report(y_test, y_pred))

    # print ("\n混淆矩阵:")
    # print (metrics.confusion_matrix(y_test, y_pred))
params = {'learning_rate': 0.3,
 'max_depth': 3,
 'max_features': 7,
 'min_samples_split': 7,
 'n_estimators': 140,
 'subsample': 0.8762782548767177}
clf = GradientBoostingClassifier(**params)
train_and_evaluate(clf, X_train, X_test, y_train, y_test)

训练集精度:
1.0

测试集精度:
0.8709677419354839

分类识别报告:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.86      0.92      0.89        13
           3       1.00      0.86      0.92         7
           4       0.80      1.00      0.89         8
           5       1.00      0.50      0.67         2

    accuracy                           0.87        31
   macro avg       0.73      0.66      0.67        31
weighted avg       0.86      0.87      0.85        31


In [61]:
from sklearn.model_selection import cross_val_score
params = {
    'n_estimators': 300, # 弱分类器的个数
    'max_depth': 2,       # 弱分类器（CART回归树）的最大深度
    'min_samples_split': 3, # 分裂内部节点所需的最小样本数
    'learning_rate': 0.02,  # 学习率
    'loss': 'deviance',
    'subsample': 0.8,
    'random_state': 1
}
clf = GradientBoostingClassifier(**params)
scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
scores

array([0.90322581, 0.76666667, 0.8       , 0.8       , 0.66666667])

In [1]:
[1, 2, 3][1]

2

In [59]:
scores.mean()

0.793978494623656

In [63]:
import numpy as np
import pandas as pd
from data_pre import data_normalize
import warnings
warnings.filterwarnings('ignore')
d = pd.read_csv(r"D:\program\pycharm\model\Data\out.csv")
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [70]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_samples_split': 3, # 分裂内部节点所需的最小样本数
    'loss': 'deviance',
    'subsample': 0.8,
    'random_state': 1
}
clf = GradientBoostingClassifier(**params)
grid = GridSearchCV(
    estimator=clf,
    param_grid={
        'n_estimators': [10, 20, 50, 100, 150, 200, 250, 300, 350, 400],
        'max_features': [1, 2, 3],
        'max_depth': [2, 3, 4],
        'learning_rate': [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 1]
    },
    cv=5, scoring='accuracy', verbose=0, n_jobs=-1)
grid.fit(X, y)
print(grid.best_score_)
print(grid.best_params_)

0.8341935483870968
{'learning_rate': 0.1, 'max_depth': 4, 'max_features': 1, 'n_estimators': 250}


In [66]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = {'n_estimators': [10, 20, 50, 100, 150, 200, 250, 300, 350, 400],
              'max_depth': [2, 3, 4],
              'learning_rate': [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 1],
              'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
params = {
    'min_samples_split': 3, # 分裂内部节点所需的最小样本数
    'loss': 'deviance',
    'subsample': 0.8,
    'random_state': 1
}
clf = GradientBoostingClassifier(**params)
random = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=20, cv=5)
random.fit(X, y)
print(random_search.best_score_)
print(random_search.best_params_)

NameError: name 'random_search' is not defined

In [67]:
print(random.best_score_)
print(random.best_params_)

0.8341935483870968
{'n_estimators': 250, 'max_features': 1, 'max_depth': 4, 'learning_rate': 0.1}


In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import hyperopt
def hyperopt_object(params):
    clf = GradientBoostingClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']) + 2,
        min_samples_split=int(params['min_samples_split']) + 1,
        learning_rate=params['learning_rate'],
        max_features=int(params['max_features']) + 1,
        loss=params['loss'],
        subsample=params['subsample'],
        random_state=params['random_state']
    )
    scores = np.mean(cross_val_score(clf, X, y, scoring='accuracy', cv=5))
    return 1 - scores
params = {
    'n_estimators': hyperopt.hp.quniform("n_estimators", 100, 300, 10), # 弱分类器的个数
    'max_depth': hyperopt.hp.randint('max_depth', 15),       # 弱分类器（CART回归树）的最大深度
    'min_samples_split': hyperopt.hp.randint('min_sampes_split', 10), # 分裂内部节点所需的最小样本数
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),  # 学习率
    'max_features': hyperopt.hp.randint('max_features', 10),
    'loss': 'deviance',
    'subsample': hyperopt.hp.uniform('subsample', 5e-1, 1),
    'random_state': 1
}
trials = hyperopt.Trials()
res = hyperopt.fmin(
    hyperopt_object,
    space=params,
    algo=hyperopt.tpe.suggest,
    max_evals=20,
    trials=trials)
res

100%|██████████| 20/20 [01:08<00:00,  3.42s/trial, best loss: 0.13311827956989242]


{'learning_rate': 0.2913217213779239,
 'max_depth': 1,
 'max_features': 6,
 'min_sampes_split': 6,
 'n_estimators': 140.0,
 'subsample': 0.8762782548767177}

In [15]:
res['learning_rate'] = 0.3
res['max_depth'] = res['max_depth'] + 2
res['min_samples_split'] = res['min_sampes_split'] + 1
res['max_features'] = res['max_features'] +1
res

{'learning_rate': 0.3,
 'max_depth': 5,
 'max_features': 7,
 'min_sampes_split': 6,
 'n_estimators': 140.0,
 'subsample': 0.8762782548767177,
 'min_samples_split': 7}

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from model.NGA import NGA
def hyperopt_object(params):
    clf = GradientBoostingClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']) + 2,
        min_samples_split=int(params['min_samples_split']) + 1,
        learning_rate=params['learning_rate'],
        max_features=int(params['max_features']) + 1,
        loss=params['loss'],
        subsample=params['subsample'],
        random_state=params['random_state']
    )
    scores = np.mean(cross_val_score(clf, X, y, scoring='accuracy', cv=5))
    return 1 - scores

In [26]:
import datetime
import time
d1 = datetime.datetime.now()
time.sleep(2)
d2 = datetime.datetime.now()
(d2 - d1).seconds

2